https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [3]:
# necessary imports
import os, json
import pandas as pd
import numpy as np
import glob
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)
import re
from nltk.corpus import stopwords
stopwords = stopwords.words('german')
import spacy
nlp = spacy.load('de_dep_news_trf')
import nltk
import gensim
from gensim import corpora, models
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read lda_tweets
df = pd.read_csv('../../data/BTW17_Twitter/lda/lda_tweets.csv')
df.drop(columns='Unnamed: 0', axis=1, inplace=True)
df.head()

,created_at,id,text,user,entities,tags
0,2017-06-28,8.798635e+17,RT @Junge_Freiheit: AfD-Vize Beatrix von Storc...,"{'id': 728326774027919360, 'id_str': '72832677...","{'hashtags': [{'text': 'Ehefueralle', 'indices...",ehefueralle
1,2017-06-28,8.798879e+17,RT @AfD_Bund: Dr. @Alice_Weidel:\n»#Ehefuerall...,"{'id': 1222045254, 'id_str': '1222045254', 'na...","{'hashtags': [{'text': 'Ehefueralle', 'indices...",ehefueralle
2,2017-06-28,8.798901e+17,"RT @hessenSPD: Abstimmen, abstimmen, abstimmen...","{'id': 2443895924, 'id_str': '2443895924', 'na...","{'hashtags': [{'text': 'ehefueralle', 'indices...",ehefueralle
3,2017-06-28,8.798903e+17,RT @Die_Gruenen: Hallo @spdbt! Jetzt hindert E...,"{'id': 12302322, 'id_str': '12302322', 'name':...","{'hashtags': [{'text': 'Ehefueralle', 'indices...",ehefueralle
4,2017-06-28,8.798903e+17,"RT @Die_Gruenen: ""Wer die Debatte auch nur hal...","{'id': 391983270, 'id_str': '391983270', 'name...","{'hashtags': [{'text': 'Ehefueralle', 'indices...",ehefueralle


In [21]:
tqdm.pandas()

accepted_pos = ['NOUN', 'PROPN', 'ADJ', 'ADV', 'VERB']

# nlp pipeline for text
df['tokens'] = df['text'].progress_apply(lambda x: [token.lemma_ for token in nlp(x) if token.pos_ in accepted_pos])
       
# stopword removal
df['tokens'] = df['tokens'].apply(lambda x: [item for item in x if item not in stopwords])

# remove short words
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if len(word) >= 3])

# lower all strings
df['tokens'] = df['tokens'].apply(lambda x: [word.lower() for word in x])

In [26]:
print('original document: ')
words = []
for word in df['text'][0].split(' '):
    words.append(word)
print(words)
print('\n\ntokenized and lemmatized document: ')
print(df['tokens'][0])

original document: 
['RT', '@Junge_Freiheit:', 'AfD-Vize', 'Beatrix', 'von', 'Storch', 'zur', '#Ehefueralle:', 'Merkel', 'hat', 'rechten', 'Flügel', 'der', 'CDU', 'fast', 'widerstandslos', 'ausradiert', 'https://…']


tokenized and lemmatized document: 
['afd-vize', 'beatrix', 'storch', 'ehefueralle', 'merkel', 'recht', 'flügel', 'cdu', 'fast', 'widerstandslos', 'ausradieren']


In [61]:
hashtag_list = df['tags'].unique().tolist()
topic_word_list = []

for index in tqdm(range(len(hashtag_list))):
    hashtag = hashtag_list[index]
    
    # filter df
    temp_df = df[df['tags']==hashtag]
    
    # get dictionary
    dictionary = gensim.corpora.Dictionary(temp_df['tokens'])
        
    # get bag of words corpus
    bow_corpus = [dictionary.doc2bow(doc) for doc in temp_df['tokens']]
    
    # lda_model
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=1, id2word=dictionary, passes=2, workers=2)
    
    # get words
    x = lda_model.show_topics()
    topic_words = []
    y = x[0][1].split('"')
    for i in range(len(y)):
        if i%2 != 0:
            topic_words.append(y[i])
    
    topic_word_list.append(topic_words)
    
# save hashtag and words to output df 
output_df = pd.DataFrame({'hashtag': hashtag_list, 'topic_words': topic_word_list})

ehefueralle
gesundheit
afd
btw17
btw2017
bundestagswahl
cdu
fdp
gabriel
glyphosat
grüne
grünen
islam
kinderarmut
mutludirekt
rente
spd
afdwählen
btw
dresden
grueneversenken
pflege
stegner
freiheit
mahe
traudichdeutschland
darumgrün
denkenwirneu
darumgruen
dielinke
fakenews
inklusion
linke
merkelmussweg
noafd
tegel
tegelschliessen
gruenen
grün
sed
gez
fedidwgugl
deutschland
gruene
hütter
integration
seehofer
nonazis
zeitfuermartin
dieselgate
erdogan
europa
freedeniz
groko
klimaschutz
lindner
merkel
schulzzug
türkei
union
antifa
bundesregierung
gerechtigkeit
trump
wahlkampf
tzt
energiewende
islamisierung
mindestlohn
umwelt
demokratie
leipzig
zukunftwirdausmutgemacht
zukunft
arbeit
csu
schulz
zdf
populismus
russland
ehefüralle
brexit
maas
hamburg
mietpreisbremse
nrw
r2g
zensur
bildung
flüchtlinge
nahles
berlin
gauland
martinschulz
diesel
köln
terror
digitalisierung
demaizière
fail
elektromobilität
bundestag
klimawandel
nato
ltnrw
antisemitismus
steineke
unwählbar
altersarmut
schäuble
txl


In [63]:
output_df.head()

,hashtag,topic_words
0,ehefueralle,"[ehefueralle, @volker_beck, @beatrix_vstorch, ..."
1,gesundheit,"[gesundheit, pflege, neues, dossier, prekär, u..."
2,afd,"[afd, traudichdeutschland, @afd, @joerg_meuthe..."
3,btw17,"[btw17, afd, @afd, wählen, traudichdeutschland..."
4,btw2017,"[btw2017, traudichdeutschland, @fraukepetry, k..."


In [64]:
# save to csv
path_file = '../../data/BTW17_Twitter/lda/topics.csv'
output_df.to_csv(path_file)